LIBRERIAS A USAR

In [15]:

# Pandas para manejo de datos
import pandas as pd

# Request para hacer peticiones HTTP
import requests

# MatplotLib para grafos
import matplotlib.pyplot as plt

CARGAR DATOS

In [16]:
#API de datos abiertos de Colombia
url = "https://www.datos.gov.co/resource/75fz-q98y.json"

# Realizar la petición HTTP
response = requests.get(url)

# Verificar si la petición fue exitosa
if response.status_code == 200:

    # Convertir la respuesta en un DataFrame de Pandas
    data = response.json()
    df = pd.DataFrame(data)

else:
    print("Error al obtener los datos: ", response.status_code)

EXPLORAR DATOS

In [17]:

# Mostra el tipo de dato de cada columna
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   orden                   1000 non-null   object
 1   armas_medios            1000 non-null   object
 2   barrios_hecho           1000 non-null   object
 3   latitud                 1000 non-null   object
 4   longitud                1000 non-null   object
 5   zona                    1000 non-null   object
 6   nom_comuna              1000 non-null   object
 7   ano                     1000 non-null   object
 8   mes                     1000 non-null   object
 9   dia                     1000 non-null   object
 10  dia_semana              1000 non-null   object
 11  descripcion_conducta    1000 non-null   object
 12  conducta                1000 non-null   object
 13  clasificaciones_delito  1000 non-null   object
 14  curso_de_vida           1000 non-null   object
 15  estad

In [44]:
# Mostrar las primeras filas del DataFrame
df.head(10)

,orden,armas_medios,barrios_hecho,latitud,longitud,zona,nom_comuna,ano,mes,dia,dia_semana,descripcion_conducta,clasificaciones_delito,movil_agresor,movil_victima
0,1,ARMA BLANCA / CORTOPUNZANTE,BUENOS AIRES,"7,170,557,382","-73,135,108",URBANA,14. MORRORICO,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
1,2,ARMA BLANCA / CORTOPUNZANTE,CAMPO HERMOSO,"7,120,645,358","-7,312,605",URBANA,05. GARCÍA ROVIRA,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
2,3,ARMA BLANCA / CORTOPUNZANTE,CAMPO HERMOSO,"7,120,645,358","-7,312,605",URBANA,05. GARCÍA ROVIRA,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
3,4,ARMA BLANCA / CORTOPUNZANTE,COMUNEROS,"715,135,927","-73,145,704,583",URBANA,03. SAN FRANCISCO,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
4,5,ARMA BLANCA / CORTOPUNZANTE,GIRARDOT,"7,170,557,382","-73,135,108",URBANA,04. OCCIDENTAL,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
5,6,ARMA BLANCA / CORTOPUNZANTE,GIRARDOT,"7,170,557,382","-73,135,108",URBANA,04. OCCIDENTAL,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
6,7,ARMA BLANCA / CORTOPUNZANTE,LOS ANGELES,"7,187,455,129","-73,131,726,904",URBANA,02. NOR ORIENTAL,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
7,8,ARMA BLANCA / CORTOPUNZANTE,NARIÑO,"715,655,391","-73,140,753,417",URBANA,04. OCCIDENTAL,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
8,9,ARMA BLANCA / CORTOPUNZANTE,PROVENZA,"7,120,001,072","-73,116,084,261",URBANA,10. PROVENZA,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE
9,10,ARMA BLANCA / CORTOPUNZANTE,SOTOMAYOR,"7,161,314,229","-73,139,956,655",URBANA,12. CABECERA DEL LLANO,2010,01. ENERO,1,05. VIERNES,ARTÍCULO 111. LESIONES PERSONALES,LESIONES NO FATALES,A PIE,A PIE


In [53]:
for column in df.columns:
    num_distinct_values = len(df[column].unique())
    print(f"{column}: {num_distinct_values} distintos valores")

orden: 1000 distintos valores
armas_medios: 14 distintos valores
barrios_hecho: 177 distintos valores
latitud: 161 distintos valores
longitud: 160 distintos valores
zona: 3 distintos valores
nom_comuna: 21 distintos valores
ano: 2 distintos valores
mes: 3 distintos valores
dia: 31 distintos valores
dia_semana: 7 distintos valores
descripcion_conducta: 20 distintos valores
clasificaciones_delito: 3 distintos valores
movil_agresor: 8 distintos valores
movil_victima: 8 distintos valores


In [54]:
df.armas_medios.unique()

#NaN No reporta

array(['ARMA BLANCA / CORTOPUNZANTE', 'ARMA DE FUEGO', 'CONTUNDENTES',
       'MOTO', 'NO REPORTADO', 'SIN EMPLEO DE ARMAS', 'VEHICULO',
       'LLAVE MAESTRA', 'CORTANTES', 'LLAMADA TELEFONICA',
       'CORTOPUNZANTES', 'ESCOPOLAMINA', 'DIRECTA', 'REDES SOCIALES'],
      dtype=object)

In [50]:
df.zona.unique()

array(['URBANA', 'RURAL', 'OTROS'], dtype=object)

In [46]:
df.nom_comuna.unique()

# REVISAR QUE ES PARTE DE BUCARAMANGA

array(['14. MORRORICO', '05. GARCÍA ROVIRA', '03. SAN FRANCISCO',
       '04. OCCIDENTAL', '02. NOR ORIENTAL', '10. PROVENZA',
       '12. CABECERA DEL LLANO', '11. SUR', '01. NORTE',
       '09. LA PEDREGOSA', 'CORREGIMIENTO 1', '15. CENTRO',
       'CORREGIMIENTO 3', '06. LA CONCORDIA', '13. ORIENTAL', '17. MUTIS',
       '07. LA CIUDADELA', '08. SUR OCCIDENTE', '16. LAGOS DEL CACIQUE',
       'FLORIDABLANCA', 'CORREGIMIENTO 2'], dtype=object)

In [49]:
df.nom_comuna.unique().shape

(21,)

In [45]:
df.clasificaciones_delito.unique()

array(['LESIONES NO FATALES', 'LESIONES FATALES', 'VIOLENCIA SEXUAL'],
      dtype=object)

In [36]:
df.conducta.unique()

array(['LESIONES PERSONALES', 'HURTO A PERSONAS', 'HOMICIDIO',
       'HURTO A RESIDENCIAS', 'HURTO A ENTIDADES COMERCIALES',
       'VIOLENCIA INTRAFAMILIAR',
       'LESIONES CULPOSAS ( EN ACCIDENTE DE TRANSITO )',
       'ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS',
       'HURTO A MOTOCICLETAS', 'ACTOS SEXUALES CON MENOR DE 14 AÑOS',
       'ACOSO SEXUAL', 'HOMICIDIO CULPOSO ( EN ACCIDENTE DE TRÁNSITO)',
       'ACCESO CARNAL VIOLENTO', 'EXTORSIÓN',
       'ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS AGRAVACIÓN)',
       'ACTO SEXUAL VIOLENTO', 'HURTO A AUTOMOTORES',
       'SECUESTRO EXTORSIVO', 'HURTO A ENTIDADES FINANCIERAS',
       'ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR'],
      dtype=object)

In [35]:
df.conducta.unique().shape

(20,)

In [42]:
df.movil_agresor.unique()

#NaN No reporta

array(['A PIE', 'CONDUCTOR MOTOCICLETA', 'CONDUCTOR VEHICULO',
       'CONDUCTOR TAXI', 'NO REPORTA', 'PASAJERO BUS', 'PASAJERO TAXI',
       'BICICLETA'], dtype=object)

In [43]:
df.movil_victima.unique()

#NaN No reporta

array(['A PIE', 'NO REPORTA', 'CONDUCTOR MOTOCICLETA',
       'CONDUCTOR VEHICULO', 'PASAJERO TAXI', 'PASAJERO BUS',
       'CONDUCTOR TAXI', 'BICICLETA'], dtype=object)

In [38]:
df.descripcion_conducta.unique()



array(['ARTÍCULO 111. LESIONES PERSONALES',
       'ARTÍCULO 239. HURTO A PERSONAS', 'ARTÍCULO 103. HOMICIDIO',
       'ARTÍCULO 239. HURTO A RESIDENCIAS',
       'ARTÍCULO 239. HURTO A ENTIDADES COMERCIALES',
       'ARTÍCULO 229. VIOLENCIA INTRAFAMILIAR',
       'ARTÍCULO 120. LESIONES CULPOSAS ( EN ACCIDENTE DE TRANSITO )',
       'ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS',
       'ARTÍCULO 239. HURTO A MOTOCICLETAS',
       'ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS',
       'ARTÍCULO 210 A. ACOSO SEXUAL',
       'ARTÍCULO 109. HOMICIDIO CULPOSO ( EN ACCIDENTE DE TRÁNSITO)',
       'ARTÍCULO 205. ACCESO CARNAL VIOLENTO', 'ARTÍCULO 244. EXTORSIÓN',
       'ARTÍCULO 211. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS AGRAVACIÓN)',
       'ARTÍCULO 206. ACTO SEXUAL VIOLENTO',
       'ARTÍCULO 239. HURTO A AUTOMOTORES',
       'ARTÍCULO 169. SECUESTRO EXTORSIVO',
       'ARTÍCULO 239. HURTO A ENTIDADES FINANCIERAS',
       'ARTÍCULO 207. ACCESO CARNA

In [40]:
df.descripcion_conducta.unique().shape

(20,)

In [19]:
# Descripcion estadistica de los datos
df.describe()

,orden,armas_medios,barrios_hecho,latitud,longitud,zona,nom_comuna,ano,mes,dia,dia_semana,descripcion_conducta,conducta,clasificaciones_delito,curso_de_vida,estado_civil_persona,genero,movil_agresor,movil_victima
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,1000,14,177,161,160,3,21,2,3,31,7,20,20,3,7,7,3,8,8
top,1,CONTUNDENTES,CENTRO,"7,178,045,669","-73,130,224,418",URBANA,15. CENTRO,2010,01. ENERO,9,05. VIERNES,ARTÍCULO 239. HURTO A PERSONAS,HURTO A PERSONAS,LESIONES NO FATALES,05. ADULTEZ,SOLTERO,MASCULINO,A PIE,A PIE
freq,1,382,107,105,105,985,125,999,763,65,178,325,325,951,460,438,506,758,823


In [20]:
# Identificar datos null
df.isnull().sum()

orden                     0
armas_medios              0
barrios_hecho             0
latitud                   0
longitud                  0
zona                      0
nom_comuna                0
ano                       0
mes                       0
dia                       0
dia_semana                0
descripcion_conducta      0
conducta                  0
clasificaciones_delito    0
curso_de_vida             0
estado_civil_persona      0
genero                    0
movil_agresor             0
movil_victima             0
dtype: int64

In [51]:
duplicate_rows_data = df[df.duplicated()]
print("numero de filas dulicadas completamente: ", duplicate_rows_data.shape[0])

numero de filas dulicadas completamente:  0


FEATURE ENGINEERING


In [26]:
# Eliminar columnas innecesarias

# Se elimina estado_civil_persona por falta de utilidad en prediccion
df = df.drop(["estado_civil_persona"], axis=1)

KeyError: "['estado_civil_persona'] not found in axis"

In [27]:
# Se elimina curso_de_vida por falta de utilidad en prediccion
df = df.drop(["curso_de_vida"], axis=1)

In [29]:
# Se elimina genero por falta de utilidad en prediccion
df = df.drop(["genero"], axis=1)

In [41]:
# Se elimina conducta por redundancia con descripcion_conducta
df = df.drop(["conducta"], axis=1)